# 核子系の第一原理計算

この章では、核子系の第一原理計算についてとくにpost-HF法に着目して解説する。  

少数系の方法(Faddeev-Yakubovsky方程式、Green's function Monte Carlo法など)や、Full-CI法などについてはカバーしない(できない)。

post-HF法のうちとくにIn-medium Similarity Renormalization Group(IM-SRG)法は、
原子核の基底状態の第一原理計算手法としてだけでなく、
殻模型の有効相互作用(バレンス空間の有効核力)の導出にも用いられるなど、応用上非常に重要な手法である。

この章では、NuclearToolkit.jlを用いて、IMSRG法による核子系の第一原理計算を行う。

In [2]:
using NuclearToolkit

## In-medium Similarity Renormalization Group (IM-SRG)

核力のソフト化の議論で登場した相似変換くりこみ群(SRG)は、核力をソフト化する方法として考案された後、
核子系の第一原理計算にも応用されるようになった。

- K.Tsukiyama et al., [Phys. Rev. Lett. 106, 222502 (2011)](https://doi.org/10.1103/PhysRevLett.106.222502).
- K.Tsukiyama et al., [Phys. Rev. C 85, 061304(R) (2012)](https://doi.org/10.1103/PhysRevC.85.061304).

SRGの背景にあるアイデアは、高運動量-低運動量間のdecouplingであった。
IM−SRG法は、SRG変換のdecouplingの対象を核子多体系のparticle-hole空間に拡張したものである。

変換後のハミルトニアン$H(s)$のHF基底状態による期待値$\langle HF| H(s) | HF \rangle$が
占有状態(hole)と非占有状態(particle)間のcouplingの寄与をdecouplingするようなSRG変換を行う。
これにより、particle-hole間のcouplingは、自然な形で、hole-hole(particle-particle)間の多体Hamiltonianに
繰り込まれるため、占有状態($P$ space)のみをもちいて、ハミルトニアンの期待値(すなわち基底状態エネルギー)を計算できる。

$$
E_\mathrm{HF} \equiv \langle HF| H(0) | HF \rangle > E_\mathrm{IMSRG} \equiv \langle HF| H(s) | HF \rangle  \approx E_\mathrm{exact}
$$


schematic imageを書くと以下のようになる:

<img src="https://github.com/SotaYoshida/Lecture_CompPhys_SpSchool24/blob/main/notebooks/pic/schematic_IMSRG.png?raw=true" width="90%">

元の図は、S.R.Stroberg et al., [Ann. Rev. Nucl. and Part. Sci., 69 307-362 (2019)](https://doi.org/10.1146/annurev-nucl-101917-021120)から引用した。  
左側が変換前のハミルトニアンを表し、$P$($Q$) spaceがそれぞれ、占有状態(hole), 非占有状態(particle)に対応する。


※1. IMSRG法はvariationalでないためエネルギーの上限を与えているとは限らないが、少なくとも
particle-hole間のcouplingを考慮した有効ハミルトニアンの期待値を計算しているため、単純なHFよりも正確なエネルギーを与えることが期待される。

※2. 上ではHF基底状態としたが、一般のreference stateに対して定義される。
量子化学などで用いられるNatural Orbitals (NO)を用いる計算もあれば、multi-referenceを考える場合などもある。


IM-SRGのflow方程式は、以下のように書ける:

$$
\frac{dH(s)}{ds} = [\eta(s), H(s)]
$$

ここで、$\eta(s)$は、flowの生成子であり、

$$
H(s) = U(s) H(0) U^\dagger(s)
$$

$$
\eta(s) = \frac{dU(s)}{ds} U^\dagger(s) = -\eta^\dagger(s)
$$

を満たす、反エルミート行列である。この$\eta$についても幾らかの定義があるが、最もよく用いられるものはarctangent関数を用いたものである。
さらに詳細が知りたい人はレビュー論文[S.R. Stroberg et al., 	Ann. Rev. Nucl. and Part. Sci., 69 307-362 (2019)](https://doi.org/10.1146/annurev-nucl-101917-021120)を参照すると良い。


#### $\clubsuit$ IMSRGのMagnus展開

以下はテクニカルなので、原子核の第一原理計算を専門にしようと思う人以外は飛ばしてもよい。

実際のコードの実装では、Magnus展開を用いたIMSRGの定式化[T.D. Morris et al., Phys. Rev. C 92, 034331 (2015)](https://doi.org/10.1103/PhysRevC.92.034331)を用いることで、演算子ごとのflow方程式を考えることなく系統的に計算を行うことができる。
そのキモは、ユニタリ変換の生成子$\Omega(s)$を明示的に導入することである。

$$
U(s) & = e^{\Omega(s)},\\
O(s) & = e^{\Omega(s)}O(0)e^{-\Omega(s)}
$$

$$
\frac{d\Omega}{ds} & = \sum^\infty_{k=0} \frac{B_k}{k!} ad^{(k)}_{\Omega}(\eta), \label{eq:adjoint} \\
ad^{(k)}_{\Omega}(\eta) & = \left[ \Omega, ad^{(k-1)}_\Omega(\eta) \right],\\
ad^{(0)}_{\Omega}(\eta) & = \eta,
$$

ここで、$B_k$はBernoulli数である。

これにより、$\Omega(s)$さえ求まれば、Hamiltonianのflowだけでなく、任意の演算子のflowを計算することができる。
演算子のflowはBaker-Campbell-Hausdorff (BCH)公式を用いて計算してやればよい。


## Normal ordering (編集中)

３体力の効果を含めた多体計算手法では、実用上、通常、3体項を顕わに扱うことはできない。
そこで、３体力の効果を可能な限り取り込むために、通常、normal orderingと呼ばれる手法が用いられる。
つまり、適当なreference stateを用いて、ハミルトニアンをnormal orderingすることで、3体項の効果を2体項に取り込む。

一般的な形で、ハミルトニアンは以下のように書ける:

$$
H = (1- \frac{1}{A}) T + T^{(2)} + V^{(2)} + V^{(3)}
$$

$T$は運動量項(kinetik term)で、$V^{(2)}$は2体力、$V^{(3)}$は3体力である。
ここで、normal orderingを行うと、Hamiltonianは以下のように書きなおせる:


$$
H = E + \sum_{pq} f_{pq} :a^\dagger_p a_q: 
+ \frac{1}{4} \sum_{pqrs} \Gamma_{pqrs} a^\dagger_p a^\dagger_q a_s a_r:
+ \frac{1}{36} \sum_{pqrstu} W_{pqrstu}  a^\dagger_p a^\dagger_q a^\dagger_r a_u a_t a_s:
$$

それぞれ(0,1,2,3-body)の寄与は、

$$
E = (1- \frac{1}{A}) \sum_a \langle a | T | a \rangle n_a 
+ \frac{1}{2} \sum_{ab} \langle ab | T^{(2)}+ V^{(2)} | ab \rangle n_a n_b 
+ \frac{1}{6} \sum_{abc} \langle abc | V^{(3)} | abc \rangle n_a n_b n_c \\
f_{pq} = \langle p | T | q \rangle + \sum_{a} \langle pa | V^{(2)} | qa \rangle n_a
+ \frac{1}{2} \sum_{ab} \langle pab | V^{(3)} | qab \rangle n_a n_b \\
\Gamma_{pqrs} = \langle pq | T^{(2)} + V^{(2)} | rs \rangle + \sum_{a} \langle pqa | V^{(3)} | rsa \rangle n_a \\
W_{pqrstu} = \langle pqr | V^{(3)} | stu \rangle
$$

上記のIMSRG計算においても、明示的に3体項$W$を扱うことは通常せず、2体項のみを扱うことが多い。これをIMSRG(2)と呼ぶ。

## Valence-space IMSRG (VS-IMSRG)

IMSRG法は、閉殻やsub-shell closedの核子系の基底状態の計算には非常に有効であるが、odd核など、開殻の計算には適さない。
そのため、開殻系の計算には、Valence-space IMSRG (VS-IMSRG)法が用いられる。

IMSRG法のユニタリ変換のdecouplingの対象が、particle-hole間であったのに対し、
VS-IMSRG法は、valence spaceとcore space(ないし模型空間外の軌道)の間のdecouplingを行う。
通常、この手続きは、IMSRG flowのあとにVS-IMSRG flowを行うことで実現される。

こうして得られたVS-IMSRGのハミルトニアン$H(s)$の1体・2体の行列要素を、殻模型の有効相互作用として用いることができる。

伝統的には、殻模型計算のインプットである有効相互作用は、現象論的なフィットの他、
$G$-matrixを用いた方法や多体摂動論(MBPT)が用いられてきたが、VS-IMSRG法を用いた方法は、
有効演算子の導出も可能であるという点で、従来の方法とは一線を画した性質を持つ。

たとえば、β崩壊の遷移強度を計算する場合、エネルギースペクトルを系統的に記述するよう構成された現象論的相互作用では、
quenching factorといって、模型空間外の軌道の効果をパラメータ化する必要があるが、
VS-IMSRG法を用いた方法では、模型空間外の軌道の効果をVS-IMSRGのユニタリ変換によって取り込むことができるため、
そうしたパラメータ化を行う必要がないと期待されている。  
P.Gysbers et al., [Nature Physics 15, 428–433 (2019)](https://doi.org/10.1038/s41567-019-0450-7)



## $\clubsuit$ HF-MBPT 

`NuclearToolkit.jl`では、いくつかの多体計算手法が実装されているが、
(sub-)shell-closedな原子核の場合、その基底状態のエネルギーと半径のみに興味がある場合、
最もカンタンな方法は、HF-MBPT (Hartree-Fock Many-Body Perturbation Theory)と呼ばれる計算によるものである。

Hartree-Fock (HF)の基底状態をゼロ次近似として、多体摂動論に基づいてスカラー演算子の期待値の補正項を計算する方法で、
量子化学では、Møller–Plesset法とも呼ばている(完全に同一のものを指すのかは著者は理解していない)。

Energyに対するfirst-orderの補正はby definitionでゼロとなり、高次の補正を考えることになる。
NuclearToolkit.jlでは、2次と3次の補正を考えることができる。

Many-Body Perturbation Theory (MBPT)やCoupled-Cluster法などの多体計算手法の詳細については、
Isaiah Shavitt and Rodney J. Bartlettによる有名な黒い本を参照されたい:  
[Many-Body Methods in Chemistry and Physics: MBPT and Coupled-Cluster Theory (Cambridge Molecular Science)](https://doi.org/10.1017/CBO9780511596834)

核力とMBPTの性質上、相互作用や原子核によっては結果が上手く収束しない場合もあるが、
十分にソフトな相互作用であれば、HF-MBPT(3)を用いれば
Coupled-Cluster法のCCSDやIM-SRG(2)と*そう遠くない*エネルギーの推定値を得ることができる。




## NuclearToolkit.jlを用いたIMSRG計算

コード中のインターフェースはHF-MBPTとIMSRG法で共通になっており、`hf_main`関数が主な関数となる。
(IMSRG法で計算する/しない場合のいずれでもHF-MBPTの計算を試みるようになっている)  
`doIMSRG`というオプション変数を`true`にすることで、IMSRG法の計算を行うことができる。

In [3]:
sntf = "tbme_em500n3lo_srg2.0hw20emax4.snt.bin"
hw = 20.0
emax = 4
hf_main(["O16"],sntf,hw,emax; doIMSRG=true)

Since optional_parameters.jl is not found, the default parameters will be used.
You can specify the parameters with optional argument, fn_params like make_chiEFTint(;fn_params="PATH_TO_YOUR_FILE").


size of dWS (jmax 9 lmax 40 e2max 8 Nnmax 20):
     dtri    4.46 MB    dcgm0    1.11 MB  d6j_int    1.11 MB   d6j_lj    0.28 MB
  d9j_lsj    1.76 MB  dictHOB    0.44 MB
target: O16 Ref. => Z=8 N=8 

E:    -132.7334  = E1b    363.65  + E2b   -496.38   (    -55.15  -367.06   -74.17)
EMP2    -23.70205 1b    -0.000 pp    -1.437 pn   -20.810 nn    -1.455


EMP3      0.37877 pp    -0.110 hh    -1.895 ph     2.384
E_HF   -132.73341  E_MBPT(3) =    -156.0567  Eexp:     -127.619

Since optional_parameters.jl is not found, the default parameters will be used.
def-by-run d6j_lj done! 

4191
 step:        s             E0   ||Omega_1||   ||Omega_2||     ||Eta_1||     ||Eta_2||    Ncomm.  nwritten
     0    0.000  -132.73340718  0.000000e+00  0.000000e+00  6.611426e-17  1.410541e+00         0         0


...

    87   87.000  -156.59032164  1.387372e-01  1.879420e-01  3.825111e-07  1.259474e-06       663         2


    88   88.000  -156.59032164  1.387376e-01  1.879430e-01  3.417405e-07  1.125231e-06       671         2


    89   89.000  -156.59032164  1.387378e-01  1.879439e-01  3.053155e-07  1.005296e-06       679         2


    90   90.000  -156.59032164  1.387381e-01  1.879447e-01  2.727730e-07  8.981453e-07       687         2


true

- HF: -132.951
- HF-MBPT(3): -156.057
- IMSRG(2): -156.590

実験値は-127.619 MeVである。HFの方が実験値に近いじゃないか！と思うかもしれないが、今考えている相互作用はinduced&genuine３体力の寄与を無視しているため、束縛エネルギーを過大評価するのが自然な相互作用になっている。
実験値との比較を考えてもそう意味はない。
